In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import datetime

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [2]:
# Scale the pixel values to be between 0 and 1 and reshape
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0
# Split data into 90% training and 10% validation sets
split = int(0.9 * len(x_train))
x_train, x_valid = x_train[:split], x_train[split:]
y_train, y_valid = y_train[:split], y_train[split:]

In [3]:
# Define Custom model architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [4]:
# Define tensorboard logs directory
log_dir = "C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistCustom"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [5]:
# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_valid, y_valid), callbacks=[tensorboard_callback])

Epoch 1/30
1688/1688 [==============================] - 126s 69ms/step - loss: 0.1384 - accuracy: 0.9574 - val_loss: 0.0663 - val_accuracy: 0.9812
Epoch 2/30
1688/1688 [==============================] - 108s 64ms/step - loss: 0.0464 - accuracy: 0.9854 - val_loss: 0.0448 - val_accuracy: 0.9872
Epoch 3/30
1688/1688 [==============================] - 108s 64ms/step - loss: 0.0309 - accuracy: 0.9910 - val_loss: 0.0329 - val_accuracy: 0.9917
Epoch 4/30
1688/1688 [==============================] - 106s 63ms/step - loss: 0.0224 - accuracy: 0.9924 - val_loss: 0.0331 - val_accuracy: 0.9923
Epoch 5/30
1688/1688 [==============================] - 114s 67ms/step - loss: 0.0178 - accuracy: 0.9938 - val_loss: 0.0335 - val_accuracy: 0.9913
Epoch 6/30
1688/1688 [==============================] - 108s 64ms/step - loss: 0.0122 - accuracy: 0.9962 - val_loss: 0.0430 - val_accuracy: 0.9892
Epoch 7/30
1688/1688 [==============================] - 103s 61ms/step - loss: 0.0106 - accuracy: 0.9965 - val_loss: 0

In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Generate predictions on the test set
y_pred = model.predict(x_test)

# Convert predicted probabilities to predicted class labels
y_pred_classes = np.argmax(y_pred, axis=1)

313/313 [==============================] - 3s 10ms/step


In [8]:
# Compute evaluation metrics
conf_matrix = confusion_matrix(y_test,y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='macro')
recall = recall_score(y_test, y_pred_classes, average='macro')
f1score = f1_score(y_test, y_pred_classes, average='macro')
auc_roc = roc_auc_score(y_test, y_pred, multi_class='ovr')

# Print the evaluation metrics
print("Test Loss:", test_loss)
print("Accuracy:", test_acc)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)
print("AU-ROC:", auc_roc)

Test Loss: 0.05776825547218323
Accuracy: 0.9908000230789185
Confusion Matrix:
 [[ 978    0    0    0    0    0    1    1    0    0]
 [   0 1126    1    2    0    1    5    0    0    0]
 [   2    1 1015    1    1    0    1   10    1    0]
 [   0    0    0 1004    0    3    0    1    2    0]
 [   0    0    0    0  973    0    2    0    1    6]
 [   0    0    1    7    0  881    1    1    0    1]
 [   5    2    0    0    1    1  949    0    0    0]
 [   0    3    0    0    0    1    0 1021    0    3]
 [   4    1    0    1    0    1    0    0  967    0]
 [   0    1    0    1    3    3    0    5    2  994]]
Precision: 0.990783103576997
Recall: 0.990786242493817
F1-score: 0.9907724942613649
AU-ROC: 0.9999366417214752


In [9]:
# Log evaluation metrics to TensorBoard
test_writer = tf.summary.create_file_writer(log_dir)
with test_writer.as_default():
    tf.summary.scalar('Accuracy', test_acc, step=0)
    tf.summary.scalar('Precision', precision, step=0)
    tf.summary.scalar('Recall', recall, step=0)
    tf.summary.scalar('F1-score', f1score, step=0)
    tf.summary.scalar('AU-ROC', auc_roc, step=0)

In [ ]:
# tensorboard --logdir_spec=FashionMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP",FashionLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionLeNet",FashionCustom:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionCustom",MnistLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistLeNet",MnistMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistMLP",MnistCustom:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistCustom"